In [7]:
import pandas as pd
from sqlalchemy import create_engine
import dotenv
import os

dotenv.load_dotenv()

PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DB")

# SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}?sslmode=require"
)

# Create a connection object from the engine
conn = engine.connect()

In [8]:

def get_table_names():
    query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    """
    df = pd.read_sql(query, conn)
    return df["table_name"].tolist()

get_table_names()


['eventtypes',
 'matches',
 'matchevents',
 'player_tracking',
 'players',
 'qualifiers',
 'qualifiertypes',
 'spadl_actions',
 'teams',
 'role_passwords']

In [9]:

def get_table_info(table_name):
    query = f"""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = '{table_name}'
    """
    df = pd.read_sql(query, conn)
    return df

get_table_info("matches")

,column_name,data_type
0,match_id,character varying
1,match_date,timestamp without time zone
2,home_team_id,character varying
3,away_team_id,character varying
4,home_score,integer
5,away_score,integer


In [10]:
def get_table_rows(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)
    return df
get_table_rows("matches")


,match_id,match_date,home_team_id,away_team_id,home_score,away_score
0,5oc8drrbruovbuiriyhdyiyok,2024-07-26,1oyb7oym5nwzny8vxf03szd2h,b7jmo07lqav0wfe2mtlzyspak,1,1
1,5r7lyj3frtml3aqy458jrxj4k,2024-07-28,920raeoumft3q2wmvc296uceo,7zibyotq1x1hq6pmgibhr0e6s,0,1
2,5ptnar4qtaltvcfsjdw9vzhg4,2024-07-27,6vyr13j6kbwdtxjwm1m4m8nu2,2989v3kho9h5jaboe1nr7lx21,1,0
3,5qqz2do47zjms7rfrwzcths7o,2024-07-28,1uk609mhtsfhokhaewjapb26,8y3iucyxguipljcmf87a11bk9,0,1
4,5pcyhm34h5c948yji4oryevpw,2024-07-27,cyrrlv6l1onld5x247w1q1jlr,bw9wm8pqfzcchumhiwdt2w15c,0,0
...,...,...,...,...,...,...
141,77w03jwg8cs1dkqemcax540t0,2024-12-13,b7jmo07lqav0wfe2mtlzyspak,7zibyotq1x1hq6pmgibhr0e6s,1,1
142,789p8p9wchivsno0m437haeqc,2024-12-15,2989v3kho9h5jaboe1nr7lx21,6vyr13j6kbwdtxjwm1m4m8nu2,0,2
143,78nndczlbj2214zhtbfahs2s4,2024-12-22,ecj1gmc7pqcjo4sc9hvghj5u3,1oyb7oym5nwzny8vxf03szd2h,2,2
144,791cri1kfsl6c1h5ozv0iqcyc,2024-12-22,89xivtxk59uzvhdsd6miao6tm,6vyr13j6kbwdtxjwm1m4m8nu2,2,0


In [ ]:
tables = get_table_names()

# count all rows in all tables
def get_row_count(table_name):
    query = f"SELECT COUNT(*) FROM {table_name}"
    df = pd.read_sql(query, conn)
    return df.iloc[0, 0]
row_counts = {}

for table in tables:
    row_counts[table] = get_row_count(table)
row_counts
    